In [48]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import csv
import re

In [49]:
def get_url(url): 
    page = urllib.request.urlopen(url)
#Cooking the Soup
    soup = BeautifulSoup(page,"html.parser")

    web_list=[]
#Scraping "Link" (href)
    for a in soup.find_all("a", href=True):
        web_list.append(a["href"])
    speech=[]
    for i in web_list:
        if i.startswith("speeches"):
            speech.append(i)
    speech_web=[]
    for i in speech:
        if i.endswith('htm'):
            speech_web.append(i)
    
    return speech_web

def modify_url(speech_web):
    start = speech_web.index('speeches/barackobama/barackobamainauguraladdress.htm')
    speech_president = speech_web[start:]

# remove replicate
    new = []
    for i in speech_president:
        if i not in new:
            new.append(i)

    speech_url = []
    for val in range(len(new)):
        speech_url.append("https://www.americanrhetoric.com/" + str(new[val]))    
    return speech_url

In [66]:
def extract_text(url):
        user = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
        resp = requests.get(url, headers=user)
        content = BeautifulSoup(resp.text, "html.parser")
        t = str(content.title)
        title = t[t.find("Obama") + 7:t.find("</title>")].strip()
        title = re.sub(r'\r\n', '', title)
        speech = content.get_text(strip=True)
        return title, speech
    
def useful_speech(speech):
        useful = speech[speech.find("from audio]") + len("from audio]"):speech.find("Book/CDs")].strip()    
        return useful

In [67]:
def extract_date(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,"html.parser")
    year = ["2009","2010","2011","2012","2013","2014","2015","2016","2017"]
    date_list = []
    for i in range(len(year)):
        date_list.extend(soup.find_all(string=re.compile(year[i])))
    s= ['Press Conference on 2010 Budget Sent to Congress','Announcement of 2012 Presidential Candidacy',
          'Honoring Golden State Warriors 2015 NBA Champs','Press Conference Following 2016 NATO Summit']
    for i in range(len(s)):
        date_list.remove(s[i])                         
    date = date_list[date_list.index("20 Jan 2009"):]
    date = [str(x) for x in date]
    return date

In [68]:
def main():
    url = "https://www.americanrhetoric.com/barackobamaspeeches.htm"
    speech_web=get_url(url)
    speech_url=modify_url(speech_web)
    speeches = [extract_text(url) for url in speech_url]
    date=extract_date(url)
    
    title=[]
    speech=[]
    for i in range(len(speech_url)):
        title.append(speeches[i][0])
        speech.append(useful_speech(speeches[i][1]))
       
    # write df_speech into csv file        
    df_speech = pd.DataFrame({'date': date,'title':title,'content':speech})
    df_speech.to_csv('obamaspeech.csv', index=False)

In [69]:
if __name__ == '__main__':
    main()